In [ ]:
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import pymysql
from config import ssh_username, ssh_host, ssh_password, ssh_port, mysql_database, mysql_host, mysql_password, mysql_port, mysql_username
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Establish SSH tunnel
with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_password=ssh_password,
    remote_bind_address=(mysql_host, mysql_port),
) as tunnel:
    print(f'Tunnel local bind port: {tunnel.local_bind_port}')
    print(f'Tunnel is active: {tunnel.is_active}')

    # Connect to MySQL through the tunnel
    conn = pymysql.connect(
        host=mysql_host,
        port=tunnel.local_bind_port,
        user=mysql_username,
        password=mysql_password,
        database='opencartdb',
        connect_timeout=30,  # Increase the connection timeout
    )

    orders = '''
            SELECT order_id, firstname, lastname, email, telephone, payment_city, payment_zone, payment_country, payment_method, 
            shipping_address_1, shipping_city, shipping_country, total, date_added, date_modified, design_file,
            shipping_date FROM oc_order
            '''
    
    order_product = '''
            SELECT order_product_id, order_id, product_id, name, model, quantity, price, total FROM oc_order_product

                     '''
    
    just_product = '''
            SELECT product_id, what, image FROM oc_product

    '''    
    

    orders = pd.read_sql_query(orders, conn)
    order_product = pd.read_sql_query(order_product, conn)
    just_product = pd.read_sql_query(just_product, conn)

    try:
        conn.close()
        print('conn is closed')
    except:
        print('conn still open')
    

df = pd.merge(orders, order_product, on = 'order_id')
df = pd.merge(df, just_product, on = 'product_id')


df = df[(df['firstname'] != '') & (df['lastname'] != '')].reset_index(drop = True)

mlb = [
    'Angels',
    'Astros',
    'Athletics',
    'Blue Jays',
    'Braves',
    'Brewers',
    'Cardinals',
    'Cubs',
    'Diamondbacks',
    'Dodgers',
    'Giants',
    'Indians',
    'Mariners',
    'Marlins',
    'Mets',
    'Nationals',
    'Orioles',
    'Padres',
    'Phillies',
    'Pirates',
    'Rangers',
    'Rays',
    'Red Sox',
    'Reds',
    'Rockies',
    'Royals',
    'Tigers',
    'Twins',
    'White Sox',
    'Yankees',
    'Baseball',
    'MLB'
]


# df = df.loc[df['name'].str.contains('|'.join(mlb), case=False)]
# df.to_csv('Baseball_Prospects.csv', index = False)

In [1]:
from modules.acquiring_db_emails import query_database, drop_unsubscribed


#Query the DB for the customer base that you are narrowing down to, and drop unsubscribed emails
mlb = [
    'Angels', 'Astros', 'Athletics', 'Blue Jays', 'Braves', 'Brewers', 'Cardinals', 'Cubs', 'Diamondbacks', 'Dodgers',
    'Giants', 'Indians', 'Mariners', 'Marlins', 'Mets', 'Nationals', 'Orioles', 'Padres', 'Phillies', 'Pirates',
    'Rangers', 'Rays', 'Red Sox', 'Reds', 'Rockies', 'Royals', 'Tigers', 'Twins', 'White Sox', 'Yankees', 'Baseball', 'MLB'
]

# Create the SQL query string
query = f'''
    SELECT * 
    FROM email_prospecting
    WHERE {' OR '.join([f"LOWER(product_name) LIKE '%{x.lower()}%'" for x in mlb])};
'''

# Assuming you have a function query_database defined
majestic = query_database(query)
majestic = drop_unsubscribed(majestic)

#This is then references from a csv in the sends

Tunnel local bind port: 59156
Tunnel is active: True
Connection is closed
Tunnel local bind port: 59322
Tunnel is active: True
Connection is closed


In [3]:
majestic

,order_id,firstname,lastname,email,telephone,payment_city,payment_zone,payment_country,payment_method,shipping_address_1,...,shipping_date,order_product_id,product_id,product_name,model,quantity,price,order_product_total,what,image
0,2279,Trail L,Combs,trail_combs@hotmail.com,810-543-0514,Columbus,Michigan,United States,credit card from old CP,8887 Gratiot,...,1900-01-01 00:00:00,27654,678,DISCONTINUED Adult Line Drive 2-Button Basebal...,1230P,2,33.1900,66.380,Teamwork Athletic,/image/catalog/products/1230P/Front/Adult_Line...
1,2290,Melvin,Lomax,joshua_ajj@yahoo.com,210-364-6907,Kensington,Maryland,United States,credit card from old CP,11306 Soward Dr,...,1900-01-01 00:00:00,67627,678,DISCONTINUED Adult Line Drive 2-Button Basebal...,1230P,20,27.2900,545.800,Teamwork Athletic,/image/catalog/products/1230P/Front/Adult_Line...
2,2309,Stacy,King,becky@customplanet.com,307 877 2239,Kemmerer,Wyoming,United States,credit card from old CP,408 North Allman Road,...,1900-01-01 00:00:00,34723,749,Blue-Jays MLB 2 Button Jersey - MA0180,Blue-Jays-MA0180,15,22.4100,336.150,Majestic Athletic,/image/catalog/products/Blue-Jays-MA0180/Front...
3,2331,Kevin R,Spengel,kevin@dikkers.com,630-532-7734,Wheaton,Illinois,United States,credit card from old CP,1613 Ashton Ct.,...,1900-01-01 00:00:00,23297,404,Cardinals MLB 2 Button Jersey - MA0180,Cardinals-MA0180,1,40.7700,40.770,Majestic Athletic,/image/catalog/products/Cardinals-MA0180/Front...
4,2352,Marco A,Rodriguez,marv_01@hotmail.com,619-519-4915,San Diego,California,United States,credit card from old CP,2362 Servando Ave,...,1900-01-01 00:00:00,18825,416,Pirates MLB 2 Button Jersey - MA0180,Pirates-MA0180,1,32.7700,32.770,Majestic Athletic,/image/catalog/products/Pirates-MA0180/Front/P...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946,70615,Abigail,Savage,info@highresortoutfitters.com,(505) 408-0588,Albuquerque,New Mexico,United States,Pay With Credit Cards,9677 Eagle Ranch Road Northwest,...,2024-02-07 00:00:00,129024,579,Youth V-Neck Raglan Baseball Jersey,NB3373,1,5.9200,5.920,Baseball Jersey,/image/catalog/products/NB3373/Front/5-Star-Fa...
2947,70644,Lisa,Walters,lwalters@cadaboard.org,(805) 895-5333,Santa Maria,California,United States,Pay With Credit Cards,715 E. Borges Drive,...,2024-02-22 00:00:00,129077,1049,Adult Full Button Baseball Jersey,N4184,23,28.6604,659.190,Baseball Jersey,/image/catalog/products/N4184/Front/Adult-Full...
2948,70647,Cindy,Collom,Plumbdawg24@gmail.com,(541) 680-0900,apple valley,California,United States,Cash On Delivery,14350 Iroquois rd,...,2024-02-23 00:00:00,129080,2485,Adult Pin Stripe Baseball Jersey Augusta Sport...,1685,18,26.2540,472.572,Baseball Jersey,/image/catalog/products/1685/Front/Pinstripe-F...
2949,70649,John,Cockerell,Jacockerell@gmail.com,(903) 277-3295,Little Rock,Arkansas,United States,Pay With Credit Cards,4304 valley view drive,...,2024-02-23 00:00:00,129083,721,Astros MLB Adult League Team Jerseys - Officia...,Astros-NY223,4,27.5000,110.000,Astros Baseball Jersey,https://customplanet.com/image/catalog//produc...
